In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, set_seed
from dataset import XingDataset
import json

model = AutoModelForCausalLM.from_pretrained("/root/holo/x/models/ft/Qwen3-4B-ft-full-2048左截断/checkpoint-992").to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained("../../models/Qwen3-4B")

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.77it/s]


In [2]:
d = XingDataset('../data/val.json').data

处理预训练数据: 100%|██████████| 200/200 [00:00<00:00, 5501.95 examples/s]


In [ ]:
d[0]

{'prompt': "你是一个法律助手，请根据以下案件信息预量刑标签(只有拘役和有期徒刑可以有缓刑)\n案件详情:\n**省**市**区人民法院\n刑事判决书\n（2018）* 011* 刑初 3019* 号\n公诉机关**省**市**区人民检察院。\n被告人郑*甲，男，1973 年 7 月 11 日出生于**省**市，\n身份证号码 ******************，汉族，小学文化，务工，户籍\n所在地为**省**市**县**镇**村民兴*甲。2012 年 8 月\n21 日因犯聚众斗殴罪被本院判处有期徒刑十个月，缓刑一年。现\n因本案于 2017 年 11 月 2 日被羁押，同年 12 月 7 日被逮捕。现被\n押于**市**区看守所。\n被告人梁*甲，男，1977 年 12 月 26 日出生于**省**县，\n身份证号码 ******************，汉族，初中文化，务工，户籍\n所在地为**省**县**镇**村8 组 16 号，现住**省**市**区城郊街旺城大道蜜糖苑 10 栋 1002 房。因本案于 2017\n年 11 月 2 日被羁押，同年 12 月 6 日被取保候审。\n被告人姜*甲，男，1977 年 9 月 16 日出生于**省景谷傣\n族彝族自治县，身份证号码 ******************，汉族，初中文\n化，务工，户籍所在地为**省**市景谷傣族彝族自治县碧安\n乡大寨村民委员会大寨村民小组三组 21。因本案于 2017 年 11 月\n2 日被羁押，同年 12 月 6 日被取保候审。\n**省**市**区人民检察院以*从检公诉刑诉\n1\n[2018]144 号起诉书指控被告人郑*甲、梁*甲、姜*甲犯故意\n伤害罪。本院适用刑事案件速裁程序，实行独任审判，公开开庭\n审理了本案。**市**区人民检察院指派检察员胡*甲出庭支\n持公诉。\n公诉机关指控：2017 年 11 月 2 日 13 时许，被*甲冉*甲到\n**省**市**区城郊街**村钢管厂工地仓库向被告人梁*甲讨要工钱，后双方发生争吵，继而被*甲冉*甲手持扳手与被\n告人郑*甲手持砍柴刀、姜*甲手持木棍、梁*甲用手相互殴打，\n致使双方不同程度受伤。经鉴定，被*甲冉*甲头面部、四肢损\n伤符合钝物作用形成，左背部损伤符合锐器作用形成，造成双侧\n鼻骨粉碎性骨折

: 

In [ ]:
idx = 23
model_inputs = tokenizer(d[idx]['prompt'].split('<|im_start|>')[0]+'<|im_start|>', return_tensors="pt").to(model.device)
generated_ids = model.generate(**model_inputs, max_new_tokens=1024)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 
output = tokenizer.decode(output_ids, skip_special_tokens=True).strip()

In [10]:
json.loads(output.replace("'", '"').replace('False', 'false').replace('True', 'true'))

[{'person_name': '陈*甲',
  'predicted_sentence1': {'desc': '管制（月数）', 'value': 0.0},
  'predicted_sentence2': {'desc': '拘役（月数）', 'value': 0.0},
  'predicted_sentence3': {'desc': '有期徒刑（月数）', 'value': 7.0},
  'predicted_sentence4': {'desc': '缓刑（月数）', 'value': 12.0},
  'predicted_sentence5': {'desc': '无期徒刑', 'value': False},
  'predicted_sentence6': {'desc': '死刑', 'value': False}}]

In [ ]:
json.loads(d[idx]['prompt'].split('<|im_start|>')[1].split('<|im_end|>')[0].replace("'", '"').replace('False', 'false').replace('True', 'true'))

[{'person_name': '陈*甲',
  'predicted_sentence1': {'desc': '管制（月数）', 'value': 0},
  'predicted_sentence2': {'desc': '拘役（月数）', 'value': 6.0},
  'predicted_sentence3': {'desc': '有期徒刑（月数）', 'value': 0.0},
  'predicted_sentence4': {'desc': '缓刑（月数）', 'value': 10.0},
  'predicted_sentence5': {'desc': '无期徒刑', 'value': False},
  'predicted_sentence6': {'desc': '死刑', 'value': False}}]

: 